Import python libraries required for the script

In [1]:
# Performance metrics
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix

#Classifier imports
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# Initialize our classifiers
BNB = BernoulliNB()
GNB = GaussianNB()
KNN = KNeighborsClassifier(n_neighbors=1)
LR = LogisticRegression(solver='lbfgs', multi_class='multinomial')
LSVC = LinearSVC(max_iter=2000, tol=0.001)
MLP = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1, warm_start=True)
MNB = MultinomialNB()
# NSVC = NuSVC(gamma='scale') # need to study about good value for nu
SGD = SGDClassifier(max_iter=1000, tol=0.001)
SVC = SVC(gamma='scale')


ADA = AdaBoostClassifier()
BAG = BaggingClassifier()
XTree = ExtraTreesClassifier()
GBC = GradientBoostingClassifier()
RFC = RandomForestClassifier()
# Vote = VotingClassifier()

Read the CSV files

In [3]:
df = pd.read_csv('VMI_Data_BP1_V02_AK6_mod02.csv')
print(df.columns)

Index(['CUT_LENGTH', 'CONV_WAIT_TIME', 'LFT_SPLICE_LENGTH',
       'MID_SPLICE_LENGTH', 'RHT_SPLICE_LENGTH', 'SPLICE_OK', 'BP1_S8_L',
       'BP1_S7_L', 'BP1_S6_L', 'BP1_S5_L', 'BP1_S4_L', 'BP1_S3_L', 'BP1_S2_L',
       'BP1_S1_ML', 'BP1_S1_MR', 'BP1_S2_R', 'BP1_S3_R', 'BP1_S4_R',
       'BP1_S5_R', 'BP1_S6_R', 'BP1_S7_R', 'BP1_S8_R', 'BP1_LENGTH',
       'LFT_SPLICE_DELTA', 'LFT_SPLICE_PREV', 'LFT_SPLICE_MA5',
       'LFT_SPLICE_MA10', 'LFT_SPLICE_MA20', 'LFT_SPLICE_MA50',
       'LFT_SPLICE_SLOPE5', 'LFT_SPLICE_SLOPE10', 'LFT_SPLICE_SLOPE20',
       'LFT_SPLICE_SLOPE50', 'MID_SPLICE_DELTA', 'MID_SPLICE_PREV',
       'MID_SPLICE_MA5', 'MID_SPLICE_MA10', 'MID_SPLICE_MA20',
       'MID_SPLICE_MA50', 'MID_SPLICE_SLOPE5', 'MID_SPLICE_SLOPE10',
       'MID_SPLICE_SLOPE20', 'MID_SPLICE_SLOPE50', 'RHT_SPLICE_DELTA',
       'RHT_SPLICE_PREV', 'RHT_SPLICE_MA5', 'RHT_SPLICE_MA10',
       'RHT_SPLICE_MA20', 'RHT_SPLICE_MA50', 'RHT_SPLICE_SLOPE5',
       'RHT_SPLICE_SLOPE10', 'RHT_SPLICE_SLOPE20'

In [4]:
df = df.drop(['LFT_SPLICE_LENGTH', 'MID_SPLICE_LENGTH', 'RHT_SPLICE_LENGTH', 'SPLICE_OK', 
              'LFT_SPLICE_GRADE', 'MID_SPLICE_GRADE', 'RHT_SPLICE_GRADE', 'LFT_SPLICE_DELTA', 
              'LFT_SPLICE_MA5', 'LFT_SPLICE_MA10', 'LFT_SPLICE_MA20', 'LFT_SPLICE_MA50', 
              'LFT_SPLICE_SLOPE5', 'LFT_SPLICE_SLOPE10', 'LFT_SPLICE_SLOPE20', 'LFT_SPLICE_SLOPE50',
              'MID_SPLICE_DELTA', 'MID_SPLICE_MA5', 'MID_SPLICE_MA10', 'MID_SPLICE_MA20',
              'MID_SPLICE_MA50', 'MID_SPLICE_SLOPE5', 'MID_SPLICE_SLOPE10', 'MID_SPLICE_SLOPE20',
              'MID_SPLICE_SLOPE50', 'RHT_SPLICE_DELTA',  'RHT_SPLICE_MA5', 'RHT_SPLICE_MA10', 
              'RHT_SPLICE_MA20', 'RHT_SPLICE_MA50', 'RHT_SPLICE_SLOPE5', 'RHT_SPLICE_SLOPE10',
              'RHT_SPLICE_SLOPE20', 'RHT_SPLICE_SLOPE50'], axis=1)

print(df.columns)

Index(['CUT_LENGTH', 'CONV_WAIT_TIME', 'BP1_S8_L', 'BP1_S7_L', 'BP1_S6_L',
       'BP1_S5_L', 'BP1_S4_L', 'BP1_S3_L', 'BP1_S2_L', 'BP1_S1_ML',
       'BP1_S1_MR', 'BP1_S2_R', 'BP1_S3_R', 'BP1_S4_R', 'BP1_S5_R', 'BP1_S6_R',
       'BP1_S7_R', 'BP1_S8_R', 'BP1_LENGTH', 'LFT_SPLICE_PREV',
       'MID_SPLICE_PREV', 'RHT_SPLICE_PREV', 'SPLICE_GRADE'],
      dtype='object')


Check the columns in the data

In [5]:
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
print(missing_data.head(10))
df = df.dropna()

                Total  Percent
SPLICE_GRADE        0      0.0
BP1_S1_MR           0      0.0
CONV_WAIT_TIME      0      0.0
BP1_S8_L            0      0.0
BP1_S7_L            0      0.0
BP1_S6_L            0      0.0
BP1_S5_L            0      0.0
BP1_S4_L            0      0.0
BP1_S3_L            0      0.0
BP1_S2_L            0      0.0


In [6]:
df[(df['SPLICE_GRADE']=="Good")]=2
df[(df['SPLICE_GRADE']=="OK")]=1
df[(df['SPLICE_GRADE']=="Bad")]=0
# df.head

In [7]:
X = df.drop(['SPLICE_GRADE'], axis=1)
y = df['SPLICE_GRADE']
y = y.astype('int')

fold = 1
kf = KFold(n_splits=5, random_state=None, shuffle=True)
for train_index, test_index in kf.split(X):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x1, x2 = X.iloc[train_index], X.iloc[test_index]
    y1, y2 = y.iloc[train_index], y.iloc[test_index]
    
    print(" ********** FOLD - ", fold, " ********** ")
    # Train our classifier and test predict
    BNB.fit(x1,y1)
    y2_BNB_model = BNB.predict(x2)
    print("BernoulliNB Accuracy :", accuracy_score(y2, y2_BNB_model))

    GNB.fit(x1, y1)
    y2_GNB_model = GNB.predict(x2)
    print("GaussianNB Accuracy :", accuracy_score(y2, y2_GNB_model))

    KNN.fit(x1,y1)
    y2_KNN_model = KNN.predict(x2)
    print("KNeighborsClassifier Accuracy :", accuracy_score(y2, y2_KNN_model))

    LR.fit(x1,y1)
    y2_LR_model = LR.predict(x2)
    print("LogisticRegression Accuracy :", accuracy_score(y2, y2_LR_model))

    LSVC.fit(x1,y1)
    y2_LSVC_model = LSVC.predict(x2)
    print("LinearSVC Accuracy :", accuracy_score(y2, y2_LSVC_model))

    MLP.fit(x1,y1)
    y2_MLP_model = MLP.predict(x2)
    print("MLPClassifier Accuracy :", accuracy_score(y2, y2_MLP_model))
    
    MNB.fit(x1,y1)
    y2_MNB_model = MNB.predict(x2)
    print("MultinomialNB Accuracy :", accuracy_score(y2, y2_MNB_model))
    
#     NSVC.fit(x1,y1)
#     y2_NSVC_model = NSVC.predict(x2)
#     print("NuSVC Accuracy :", accuracy_score(y2, y2_NSVC_model))
    
    SGD.fit(x1,y1)
    y2_SGD_model = SGD.predict(x2)
    print("SGDClassifier Accuracy :", accuracy_score(y2, y2_SGD_model))
    
    SVC.fit(x1,y1)
    y2_SVC_model = SVC.predict(x2)
    print("SVC Accuracy :", accuracy_score(y2, y2_SVC_model))
    
    ADA.fit(x1,y1)
    y2_ADA_model = ADA.predict(x2)
    print("AdaBoostClassifier Accuracy :", accuracy_score(y2, y2_ADA_model))
    
    BAG.fit(x1,y1)
    y2_BAG_model = BAG.predict(x2)
    print("BaggingClassifier Accuracy :", accuracy_score(y2, y2_BAG_model))

    XTree.fit(x1,y1)
    y2_XTree_model = XTree.predict(x2)
    print("ExtraTreesClassifier Accuracy :", accuracy_score(y2, y2_XTree_model))

    GBC.fit(x1,y1)
    y2_GBC_model = GBC.predict(x2)
    print("GradientBoostingClassifier Accuracy :", accuracy_score(y2, y2_GBC_model))

    RFC.fit(x1,y1)
    y2_RFC_model = RFC.predict(x2)
    print("RandomForestClassifier Accuracy :", accuracy_score(y2, y2_RFC_model))
    
    fold += 1


 ********** FOLD -  1  ********** 
BernoulliNB Accuracy : 0.9774486989634017
GaussianNB Accuracy : 1.0
KNeighborsClassifier Accuracy : 1.0
LogisticRegression Accuracy : 1.0
LinearSVC Accuracy : 0.9774486989634017
MLPClassifier Accuracy : 0.9774486989634017
MultinomialNB Accuracy : 0.7909879416120161
SGDClassifier Accuracy : 0.9774486989634017
SVC Accuracy : 1.0
AdaBoostClassifier Accuracy : 1.0
BaggingClassifier Accuracy : 1.0
ExtraTreesClassifier Accuracy : 1.0


C:\Users\SajidPatel\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


GradientBoostingClassifier Accuracy : 1.0
RandomForestClassifier Accuracy : 1.0
 ********** FOLD -  2  ********** 


C:\Users\SajidPatel\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


BernoulliNB Accuracy : 0.9781256610958324
GaussianNB Accuracy : 1.0
KNeighborsClassifier Accuracy : 1.0
LogisticRegression Accuracy : 1.0
LinearSVC Accuracy : 0.9781256610958324
MLPClassifier Accuracy : 0.9781256610958324
MultinomialNB Accuracy : 0.7946689232071081
SGDClassifier Accuracy : 0.9781256610958324
SVC Accuracy : 1.0
AdaBoostClassifier Accuracy : 1.0
BaggingClassifier Accuracy : 1.0
ExtraTreesClassifier Accuracy : 1.0
GradientBoostingClassifier Accuracy : 1.0
RandomForestClassifier Accuracy : 1.0
 ********** FOLD -  3  ********** 
BernoulliNB Accuracy : 0.9782516713209782
GaussianNB Accuracy : 1.0
KNeighborsClassifier Accuracy : 1.0
LogisticRegression Accuracy : 1.0
LinearSVC Accuracy : 0.9782516713209782
MLPClassifier Accuracy : 0.9782516713209782
MultinomialNB Accuracy : 0.7973681983582974
SGDClassifier Accuracy : 0.9782516713209782
SVC Accuracy : 1.0
AdaBoostClassifier Accuracy : 1.0
BaggingClassifier Accuracy : 1.0
ExtraTreesClassifier Accuracy : 1.0
GradientBoostingClass

In [ ]:
target_names = ['Bad', 'OK', 'Good']
class_names = target_names

In [8]:
print(" *********** GaussianNB *********** ")
print(classification_report(y2, y2_GNB_model, target_names=target_names, sample_weight=None, digits=4))

print(" *********** KNeighborsClassifier *********** ")
print(classification_report(y2, y2_KNN_model, target_names=target_names, sample_weight=None, digits=4))

print(" *********** LogisticRegression *********** ")
print(classification_report(y2, y2_LR_model, target_names=target_names, sample_weight=None, digits=4))

print(" *********** SVC *********** ")
print(classification_report(y2, y2_SVC_model, target_names=target_names, sample_weight=None, digits=4))


 *********** GaussianNB *********** 


NameError: name 'target_names' is not defined

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y2, y2_GNB_model)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='GaussianNB Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='GaussianNB Normalized confusion matrix')

plt.show();

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y2, y2_KNN_model)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='KNeighborsClassifier Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='KNeighborsClassifier Normalized confusion matrix')

plt.show();

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y2, y2_SVC_model)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='SVC Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='SVC Normalized confusion matrix')

plt.show();

In [ ]:
# Try out these
# sklearn.multioutput.MultiOutputClassifier
# sklearn.multiclass.OutputCodeClassifier
# sklearn.multiclass.OneVsOneClassifier
# sklearn.multiclass.OneVsRestClassifier
# sklearn.model_selection.RandomizedSearchCV -- https://stackoverflow.com/questions/52029408/sklearn-mlp-classifier-hyperparameter-optimization-randomizedsearchcv
# sklearn.model_selection.check_cv
# sklearn.model_selection.StratifiedKFold
# sklearn.linear_model.RidgeClassifierCV
# sklearn.linear_model.PassiveAggressiveClassifier
# sklearn.linear_model.LogisticRegressionCV
# sklearn.gaussian_process.GaussianProcessClassifier
# sklearn.tree.DecisionTreeClassifier
# sklearn.tree.ExtraTreeClassifier
# Feature significance
# Narrow to Wide Splice
# Prev Deltas MA & Slope
